In [2]:
%load_ext autoreload
%autoreload 2

import alpaca_trade_api as alp
import matplotlib.pyplot as plt
import lib.TimeKeeper as tk
import lib.Toolbox as tb
import lib.Broker as br
import lib.Scrivener as sc
import lib.Logger as log
import pandas as pd
from lib.Config import Config
import pytz

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
api = br.paper_api()

In [ ]:
#trades = api.get_trades('NVDA', tk.to_time_s(4,3,9,30),tk.to_time_s(4,3,16,0))
#pd.to_pickle(trades.df.price, './trade_data.pkl')
df = pd.read_pickle('./trade_data.pkl')

# GOAL
Our goal here is to determine whether trade prices can be used to predict immediate next step price, which basically means seasonality and trend. In otherwords, can the trade prices of the past minute tell us how the price is going to go within the next five seconds?

In [ ]:
results = []
for minute in tk.get_ideal_workday(tk.to_time(4,3))[1:-1]:
    lmf = df[minute-pd.DateOffset(minutes=1):minute]
    mf = df[minute:minute+pd.DateOffset(minutes=1)]
    pdel, cdel = lmf.diff().sum(), mf.diff().sum()
    lmf_std = lmf.std()
    mf_initial = mf.iloc[0]
    
    pred = mf.max() > mf_initial + lmf_std if pdel > 0 else mf.min() < mf_initial - lmf_std

    results.append({'minute':minute, 'prev_delta':pdel, 'cur_delta':cdel, 'predicted': pred})
results = pd.DataFrame(results)

results.set_index('minute')

In [ ]:
# Get transactions from logs
transactions = []
with open('./logs/training/2024-04-03-Wed/NVDA_TR_ORDERS.tsv', 'r') as f:
    for l in [line.split('\t') for line in f.readlines()[1:]]:
        transactions.append([tk.to_time(4,3,l[0].split(':')[0],l[0].split(':')[1]),l[2],l[3],l[6].strip()])

In [ ]:
# Test Reliability of Prediction for Transaction Points
results = []
profit, hold = 0, 0
etime, ctime = 0, -1

for tpoint in transactions:
    minute = tpoint[0]
    lmf = df[minute-pd.DateOffset(minutes=1):minute]
    mf = df[minute:minute+pd.DateOffset(minutes=1)]
    full = df[minute-pd.DateOffset(minutes=1):minute+pd.DateOffset(minutes=1)].ewm(span=5).mean()[minute:]
    pdel, cdel = lmf.diff().sum(), mf.diff().sum()
    lmf_std = lmf.std()/2
    mf_initial = mf.iloc[0]
    
    pred = mf.max() > mf_initial + lmf_std if (tpoint[1] == 'sell' or tpoint[1] == 'shortsell') else mf.min() < mf_initial - lmf_std

    tb.qp(full, 'blue'); plt.title(tpoint[1]); plt.show()
    
    match tpoint[1]:
        case 'BUY':
            hold = -mf.iloc[etime]*100
        case 'SHORTSELL':
            hold = mf.iloc[etime]*100
        case 'SELL':
            print('sell', hold + mf.iloc[ctime]*100)
            profit += hold + mf.iloc[ctime]*100
        case 'SHORTBUY':
            print('shortbuy', hold - mf.iloc[ctime]*100)
            profit += hold - mf.iloc[ctime]*100

    results.append({'minute':minute, 'type':tpoint[1], 'max':round(mf.max(),2), 'min':round(mf.min(),2), 'init_plus':round(mf_initial + lmf_std,2), 'init_minus':round(mf_initial - lmf_std,2), 'predicted': pred, 'note':tpoint[3]})
results = pd.DataFrame(results)
results.set_index('minute')

print('Total Profit', profit)

Simple idea. Mark when the squeeze momentum indicator says to buy long or short and then verify whether the price indicated by the one deviation from the initial trade price holds over the following minute to better find high or low

In [ ]:
print(results.predicted.sum()/len(results))

In [ ]:
tb.qp(df.price[tk.to_time(4,4,10,0):tk.to_time(4,4,10,1)].ewm(span=300).mean(), 'blue')

# Contemplating
Where to even begin with this? Our primary goal is to buy and sell at a better time for better results.

So, where do we even start? What is our goal? To buy low and sell high, during the windows of the minutes that our formula dictate as being minutes of interest. So how do we do that?
1. Get the logs for all of our buy and sell transactions.
2. Get the trades that went on during those times (and potentially the minut leading up to those times)
3. Develop an algorithm that is able to predict with reasonable certainty a price that can be set for this minute, or the following minute.

# Implementation of Squeeze Momentum Via Fractionation

In [ ]:
df = pd.read_pickle('./trade_data.pkl')


In [ ]:
trades = api.get_trades('NVDA', tk.to_time_s(4,8,9,30),tk.to_time_s(4,8,16,0))
df = trades.df.price
#pd.to_pickle(trades.df.price, './trade_data.pkl')

In [ ]:
ts = pd.Series(pd.date_range(df.index[0], end=df.index[-1], freq="10s"))
ndf = []
for i in range(len(ts))[:-1]:
    data = df[ts.loc[i]:ts.loc[i+1]]
    ndf.append({'date':ts.loc[i], 'open': data.iloc[0], 'close': data.iloc[-1], 'high':data.max(), 'low':data.min()})
ndf = pd.DataFrame(ndf).set_index('date')

In [ ]:
data = ndf
conf = Config()
atr = tb.get_atr(data)
bollinger = tb.get_bollinger_bands(data.close, conf.KC_WINDOW, num_stds=(conf.BOL_DEV,0,-conf.BOL_DEV))
keltner = tb.get_keltner_bands(data, conf.KC_WINDOW, conf.KC_MULT)
squeeze = tb.get_squeeze_momentum(data, keltner, bollinger, conf.KC_WINDOW, min_squeeze=conf.SQUEEZE_DURATION, offset=False)

In [ ]:
tb.qp(data.close, color = 'blue')
for i in squeeze.index:
    if squeeze.loc[i].enter_long: plt.axvline(i, color='green')
    if squeeze.loc[i].enter_short: plt.axvline(i, color='orange')

In [ ]:
tb.update_archive('NVDA')

In [ ]:
symbol = 'NVDA'
tb.get_data(symbol, start=tk.dto_time(tk.today()-pd.DateOffset(hours=12)), end=tk.dto_time(tk.today()-pd.DateOffset(hours=11, minutes=45)))

# Async Restart Testing

In [ ]:
import alpaca_trade_api as alp
import lib.Broker as Broker
import pandas as pd
from alpaca_trade_api.common import URL
from alpaca_trade_api.stream import Stream
import nest_asyncio
nest_asyncio.apply()
%autoawait asyncio

class Temp():
    def __init__(self):
        self.trades = None
        api = Broker.paper_api()

        stream = Stream(api._key_id, api._secret_key, base_url=URL(api._base_url), data_feed='sip')
        stream.subscribe_trades(self. trade_callback, 'NVDA')
        stream.run()

    async def trade_callback(self, t):
        '''On trade events sent by the brokerage, adds the trade to the relevant strategy's list of trades.'''
        if not isinstance(self.trades, pd.Series): self.trades = pd.Series(data=[t.price],index=[t.timestamp])
        else: self.trades.loc[t.timestamp] = t.price

t = Temp()

In [ ]:
import lib.TimeKeeper as tk
import lib.Broker as br

In [ ]:
len(t.trades[tk.to_time(4,11,9,56,30):tk.to_time(4,11,9,56,40)])

In [ ]:
api = br.paper_api()
len(api.get_trades('NVDA', start=tk.dto_time(tk.to_time(4,11,9,56,30)), end=tk.dto_time(tk.to_time(4,11,9,56,40))).df.price)

## Good News
It looks like we collect trades pretty decently. Let's collect them and then use get_trades to compare

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import lib.TimeKeeper as tk

t = pd.DataFrame(trades)
t = t.set_index('date')
print(len(t[tk.to_time(4,10,12,30,10):tk.to_time(4,10,12,30,20)]))


In [ ]:
len(api.get_trades('NVDA', tk.dto_time(tk.to_time(4,10,12,30,10)), tk.dto_time(tk.to_time(4,10,12,30,20)), feed='sip'))

In [ ]:
len(api.get_trades('NVDA', start=tk.dto_time(tk.now()-pd.DateOffset(seconds=10))))

# Final Parity Testing

In [ ]:
api = br.paper_api()
symbol = 'NVDA'

In [ ]:
tk.sync()
df = []
while True:
    bar = tb.get_segment(symbol, api)
    print(bar)
    if not isinstance(df, pd.DataFrame): df = bar
    else: df.loc[bar.name] = bar
    tk.sync()
    print(df)

# Goal: Entry Price Parity
We must create a method which, for each pip, can predict the entry price that we would attempt if we were to enter at that pip.

In [ ]:
log.get_log('NVDA', tk.d(5,8), 'training', 'orders')

In [ ]:
sc.get_archive('NVDA', tk.d(5,8), 'raw')

In [ ]:
'''Pseudo Code:
    Processed archive
    get raw archive
    for each processed pip get the entry price based on the raw trades
    compare to actual target prices from logs
'''

In [33]:
symbol, date = 'NVDA', tk.d(5,9)
raw, processed = sc.get_archive(symbol, date, 'raw'), sc.get_archive(symbol, date)

orders = log.get_log(symbol, date, 'brokerage', 'orders')
orders = orders[orders.status != 'uninitialized']
orders.index = pd.to_datetime(str(date)+' '+orders.index)#.tz_localize(pytz.timezone('America/New_York'))

/tmp/ipykernel_7578/1467425023.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  orders.index = pd.to_datetime(str(date)+' '+orders.index)#.tz_localize(pytz.timezone('America/New_York'))


In [41]:
# Get Entry Prices for each pip
eps = []
start, end = 10, 0
for index in processed.index:
    # Difference of two seconds
    trades = raw.loc[index-pd.DateOffset(seconds=start):index+pd.DateOffset(seconds=end)].price
    if len(trades) == 0: trades = raw.loc[index-pd.DateOffset(seconds=10):index+pd.DateOffset(seconds=end)].price

    #print(trades.index[0], trades.index[-1])
    print(index-pd.DateOffset(seconds=start),index+pd.DateOffset(seconds=end))
    eps.append({'index':index, 'price':round(trades.iloc[-1],2), 'qty':len(trades)})
eps = pd.DataFrame(eps).set_index('index')

total_diff = 0
for index in orders.index:
    order = orders.loc[index]
    ep = eps.loc[index]
    price = ep.price
    if 'BUY' in order.side: price -= .05
    else: price += .05
    target_price = order.target_price
    print(index, target_price, price, round(abs(target_price - price),2), 'QTY', ep.qty)
    total_diff += abs(target_price - price)
print(total_diff,'or',round(total_diff/len(orders),2),'per order of',len(orders),'orders')

2024-05-09 09:30:00-04:00 2024-05-09 09:30:10-04:00
2024-05-09 09:30:10-04:00 2024-05-09 09:30:20-04:00
2024-05-09 09:30:20-04:00 2024-05-09 09:30:30-04:00
2024-05-09 09:30:30-04:00 2024-05-09 09:30:40-04:00
2024-05-09 09:30:40-04:00 2024-05-09 09:30:50-04:00
2024-05-09 09:30:50-04:00 2024-05-09 09:31:00-04:00
2024-05-09 09:31:00-04:00 2024-05-09 09:31:10-04:00
2024-05-09 09:31:10-04:00 2024-05-09 09:31:20-04:00
2024-05-09 09:31:20-04:00 2024-05-09 09:31:30-04:00
2024-05-09 09:31:30-04:00 2024-05-09 09:31:40-04:00
2024-05-09 09:31:40-04:00 2024-05-09 09:31:50-04:00
2024-05-09 09:31:50-04:00 2024-05-09 09:32:00-04:00
2024-05-09 09:32:00-04:00 2024-05-09 09:32:10-04:00
2024-05-09 09:32:10-04:00 2024-05-09 09:32:20-04:00
2024-05-09 09:32:20-04:00 2024-05-09 09:32:30-04:00
2024-05-09 09:32:30-04:00 2024-05-09 09:32:40-04:00
2024-05-09 09:32:40-04:00 2024-05-09 09:32:50-04:00
2024-05-09 09:32:50-04:00 2024-05-09 09:33:00-04:00
2024-05-09 09:33:00-04:00 2024-05-09 09:33:10-04:00
2024-05-09 0

input_start 2024-05-09T09:45:   39   -04:00 input_end 2024-05-09T09:45:   41   -04:00 
	 2024-05-09 13:45:39.008044109+00:00 2024-05-09 13:45:40.998945959+00:00 
qty: 1801 
target price 887.52

In [34]:
api = br.paper_api()
trades = api.get_trades('NVDA', start=tk.dto_time(tk.d(5,8,15,51,10)), end=tk.dto_time(tk.d(5,8,15,51,12))).df.price
print(len(trades))

Activating with Paper
240


In [24]:
raw

,price,size
timestamp,,
2024-05-09 09:30:00.002647618-04:00,905.370,9
2024-05-09 09:30:00.003150554-04:00,905.370,50
2024-05-09 09:30:00.003150611-04:00,905.370,254
2024-05-09 09:30:00.004262818-04:00,905.370,50
2024-05-09 09:30:00.004264429-04:00,905.370,50
...,...,...
2024-05-09 11:19:07.091169888-04:00,891.900,1
2024-05-09 11:19:07.126980146-04:00,891.900,1
2024-05-09 11:19:07.182662037-04:00,891.822,1


In [29]:
sc.update_trades('NVDA', tk.today())

Activating with Paper


,price,size
timestamp,,
2024-05-09 09:30:00.002647618-04:00,905.3700,9
2024-05-09 09:30:00.003150554-04:00,905.3700,50
2024-05-09 09:30:00.003150611-04:00,905.3700,254
2024-05-09 09:30:00.004262818-04:00,905.3700,50
2024-05-09 09:30:00.004264429-04:00,905.3700,50
...,...,...
2024-05-09 11:28:52.207169549-04:00,895.3400,4
2024-05-09 11:28:52.299626827-04:00,895.4150,25
2024-05-09 11:28:52.308039948-04:00,895.5988,1


In [32]:
raw[tk.d(date.month, date.day, 11,27,40):tk.d(date.month, date.day, 11,27,50)]

,price,size
timestamp,,
2024-05-09 11:27:40.030485841-04:00,893.2300,5
2024-05-09 11:27:40.123743780-04:00,893.2800,1
2024-05-09 11:27:40.125004589-04:00,893.3688,125
2024-05-09 11:27:40.394295851-04:00,893.3800,100
2024-05-09 11:27:40.394389447-04:00,893.3900,50
...,...,...
2024-05-09 11:27:49.388595336-04:00,893.6400,1
2024-05-09 11:27:49.677361825-04:00,893.4400,1
2024-05-09 11:27:49.683288405-04:00,893.4450,30


In [ ]:
2024-05-09 11:27:51-04:00
PRICE OF THE LAST TRADE IS 893.4988 AND ITS SIZE WAS 1
pip price at 2024-05-09 11:27:52-04:00 893.5

# Goal: Exit Price Parity

In [61]:
symbol, date = 'NVDA', tk.d(5,10)
processed = sc.get_archive(symbol, date)
raw = sc.get_archive(symbol, date, 'raw')
orders = log.get_log(symbol, date, 'brokerage', 'ORDERS')

In [105]:
buffer, qty = .2, 85
total = 0
for index in orders.index:
    order, prev_close = orders.loc[index], processed.loc[index].close
    final_price = round(prev_close - buffer,2) if 'SELL' in order.side else round(prev_close + buffer,2)
    trades = raw[index + pd.DateOffset(seconds=1):index + pd.DateOffset(seconds=21)]
    if 'SELL' in order.side:
        print(trades[trades.price > final_price].price.mean(), order.filled_price)
        if final_price < trades.price.max() and qty < trades[trades.price > final_price]['size'].sum(): total += 1
    else:
        if final_price > trades.price.min() and qty < trades[trades.price < final_price]['size'].sum(): total += 1
print(total/len(orders))

'''
subdivide
    sell price max

'''

911.8991906261579 911.36
907.771539979445 907.39
904.2835852941176 903.89
898.0702938086304 897.913
898.2001366336633 897.908
896.982181893004 896.76
898.2252567126725 898.0
898.9929010619469 898.7
898.530915642458 898.157355
897.6768410794602 897.34
895.62547 0.0
895.1020576271187 894.92
897.3550554545455 897.28
897.4834910505837 897.398
899.6102255852842 899.488678
897.5426501886792 897.41
899.8772199140401 899.474298
899.0087531578947 898.831666
899.1549235772358 899.06
899.0011351955307 898.79
900.1460732394365 900.03
898.7711928160918 898.718333
898.7786727272727 898.73
898.2856644736843 898.23
898.8636973684211 898.69
897.9445136363636 897.61
897.9993460490464 897.891157
897.8149269503546 897.600667
1.0


'\nsubdivide\n    sell price max\n\n'

In [24]:
qty = 85
total = 0
for i in range(1, len(processed)-2):
    entry_price = processed.iloc[i-1].close + .5
    trades = raw[processed.index[i] + pd.DateOffset(seconds=1):processed.index[i+2] + pd.DateOffset(seconds=1)]
    if entry_price < trades.price.max(): total+=1
print(total/len(processed))

0.30440359127832406


# Thoughts on Next Steps
What are we doing? We're going to refactor a little bit, particularly we're going to probably either pair down the gambit system or scrap it entirely

Refactoring to understand what's going on and allow for a little more functionality

Introduce a system of teired buying and selling.

Goals Today
1. Refactor Gambit to be simpler and have a tiered enter and exit strategy
2. Refactor portfolio to basically not exist during brokerage testing, relying entirely on the broker

In [3]:
api = br.paper_api()

Activating with Paper


In [6]:
api.get_position('NVDA')

Position({   'asset_class': 'us_equity',
    'asset_id': '4ce9353c-66d1-46c2-898f-fce867ab0247',
    'asset_marginable': True,
    'avg_entry_price': '902.353333333',
    'change_today': '0.0039386724226173',
    'cost_basis': '2707.059999999',
    'current_price': '902.32',
    'exchange': 'NASDAQ',
    'lastday_price': '898.78',
    'market_value': '2706.96',
    'qty': '3',
    'qty_available': '3',
    'side': 'long',
    'symbol': 'NVDA',
    'unrealized_intraday_pl': '-0.099999999',
    'unrealized_intraday_plpc': '-0.0000369404442458',
    'unrealized_pl': '-0.099999999',
    'unrealized_plpc': '-0.0000369404442458'})

In [9]:
api.get_position('NVDA')

Position({   'asset_class': 'us_equity',
    'asset_id': '4ce9353c-66d1-46c2-898f-fce867ab0247',
    'asset_marginable': True,
    'avg_entry_price': '902.3675',
    'change_today': '0.0039337768975723',
    'cost_basis': '3609.47',
    'current_price': '902.3156',
    'exchange': 'NASDAQ',
    'lastday_price': '898.78',
    'market_value': '3609.2624',
    'qty': '4',
    'qty_available': '4',
    'side': 'long',
    'symbol': 'NVDA',
    'unrealized_intraday_pl': '-0.2076',
    'unrealized_intraday_plpc': '-0.0000575153692924',
    'unrealized_pl': '-0.2076',
    'unrealized_plpc': '-0.0000575153692924'})